In [1]:
from datetime import datetime 
import pandas as pd
import sqlalchemy as db
import numpy as np
import pyodbc

In [2]:
#Control execution time
start = datetime.now()

#Conection with your data base
conn = pyodbc.connect('Driver={Teradata Database ODBC Driver 16.20};'
                        'DBCNAME={10.11.240.31};'
                        'UID={APP_PlanContrGestCanales};'
                        'PWD={frBNeXosBa}')


consulta = conn.cursor()
consulta.execute('''
    CREATE VOLATILE MULTISET TABLE TEMP_TC_HISTORICO_APROBADA_ALERT AS
        (
            SELECT 	Supervisor_Cd  REG_SUPERVISOR,Supervisor_Nm SUPERVIDOR,Promotor_Cd CODPROMOT,Promotor_Nm EJECUTIVO,documento_Num NRODOCTIT,Cliente_Cd,
                    CAST(AprobacionExpedADQ_Fc AS DATE) FECRESOL,MesAprobacionExpedADQ_Fc MESRESOL,CAST(EntregaTc_Fc AS DATE) FECENTREGA ,MesEntregaTc_Dsc MESENTREGA,MESACTIVACOMISION,
                    FECACTIVACOMISION,EquipoVenta_Dsc EQUIPO,SubEquipoVenta_Dsc SUBEQUIPO,EstadoSolicitud_Cd CODESTADO,Operacion_Cd CODOPERAC,Ooperacion_Dsc OPERACION
            --SELECT *
            FROM 	DLAB_DESNEGRET.CGR_TC_HISTORICO
            WHERE 	MesAprobacionExpedADQ_Fc>=(year(date)*100+month(date)-2) AND EstadoSolicitud_Cd=3 AND Operacion_Cd IN (1,2) --and NRODOCTIT = '75489595'
            AND 	Promotor_Cd IN (SELECT REG_EJECUTIVO FROM DLAB_DESNEGRET.MAESTRO_DOTACION WHERE PERIODO=(year(date)*100+month(date)) AND EQUIPO='TLV TARJETAS' 
                                AND sub_equipo = 'EMPRENDEDOR + PREFERENTE' AND PUESTO='EJECUTIVO' )
        )

    WITH DATA 
    PRIMARY INDEX(EJECUTIVO,NRODOCTIT ) ON COMMIT PRESERVE ROWS;
        ''')
    
consulta.commit()
consulta.close()
    

query_temporal = '''
    SELECT FECRESOL, COUNT(*) CANTIDAD from TEMP_TC_HISTORICO_APROBADA_ALERT
    WHERE MESRESOL=(year(date)*100+month(date))
    AND CODESTADO=3 AND CODOPERAC IN (1,2)
    GROUP BY FECRESOL
    ORDER BY FECRESOL DESC;
    '''

data_avance = pd.read_sql_query(query_temporal,conn)

    #Description of mail
descripton='''
    <p>
    Hola team, se reporta la actualizacion por dia de las TC Aprobadas - Canal TLV Tarjetas:
    </p>
    <br>
    '''

#Head of table in HTML
head= '''       
        <table width="520" style=" border-collapse: collapse;text-align: center;" border="0">
            <thead>
                <tr>
                    <th style=" font-family:Arial;font-size:12px;background-color:#05334D;color:white" colspan="2" scope='colgroup'>TC Aprobadas</th>
                    <th style=" font-family:Arial;font-size:12px;background-color:#05334D;color:white" colspan="2" scope='colgroup'>TC Activas</th>
                </tr>
                <tr>
                    <th style=" font-family:Arial;font-size:12px" scope='col'>Fecha</th>
                    <th style=" font-family:Arial;font-size:12px" scope='col'>Cantidad</th>
                    <th style=" font-family:Arial;font-size:12px" scope='col'>Fecha</th>
                    <th style=" font-family:Arial;font-size:12px" scope='col'>Cantidad</th>
                </tr>
                <!--<tr style="height:10px;"></tr>-->
            '''



C:\Users\B41319\AppData\Local\Temp\ipykernel_21144\3413317249.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_avance = pd.read_sql_query(query_temporal,conn)


In [5]:
data_avance['FECRESOL'] = data_avance['FECRESOL'].astype(str)

In [6]:
#Convert the dataframe to numpy
df_list = data_avance.to_numpy().tolist()

In [7]:
df_list

[['2023-09-19', 4],
 ['2023-09-18', 9],
 ['2023-09-16', 2],
 ['2023-09-15', 183],
 ['2023-09-14', 236],
 ['2023-09-13', 205],
 ['2023-09-12', 79],
 ['2023-09-11', 177],
 ['2023-09-08', 219],
 ['2023-09-07', 268],
 ['2023-09-06', 274],
 ['2023-09-05', 255],
 ['2023-09-04', 216],
 ['2023-09-02', 117],
 ['2023-09-01', 278]]